In [493]:
%load_ext autoreload
%autoreload 2

from dataloader import *
import tensorflow as tf
from model import *
from tqdm import tqdm
import numpy as np
from utils import *

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import binary_accuracy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [494]:
crossval_results = []
n_splits = 1
batch_size = 4

for i in range(n_splits):
    print(f"Experiment {i+1} of {n_splits}")
    train_loader, test_loader = random_split(data, train_epochs=5, batch_size=batch_size)

    train_loss, train_acc, test_loss, test_acc = evaluate(train_loader, test_loader,
                                                 channels=64, batch_size=batch_size,
                                                 dataset="FakeNewsNet")

    crossval_results.append((train_loss, train_acc, test_loss, test_acc))

Experiment 1 of 1
Ep. 1 - Loss: 7.647. Acc: 0.504
Ep. 2 - Loss: 7.647. Acc: 0.504
Ep. 3 - Loss: 7.974. Acc: 0.483
Ep. 4 - Loss: 7.190. Acc: 0.534


ValueError: operands could not be broadcast together with shapes (4,4) (3,3) 

In [491]:
def train_step(model, loss_fn, optimizer, inputs, target):
    with tf.GradientTape() as tape:
        pred = model(inputs[:2])
        loss = loss_fn(target, pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    correct = (target.reshape(-1,1) == pred).numpy().sum()
    return loss, correct

In [492]:
def evaluate(train_loader, test_loader, num_classes=1, lr=0.001, channels=64,
             batch_size=32, dataset="FakeNewsNet"):
    if dataset == "WICO":
        num_classes = 3

    # model = FirstNet(num_classes=num_classes, channels=channels, batch_size=batch_size)
    model = FirstNet()
    loss_fn = BinaryCrossentropy()
    optimizer = Adam(learning_rate=lr)

    epoch = step = 0
    train_losses = []
    train_accuracies = []
    total = 0

    for inputs, target in train_loader:
        if target.shape[0] == batch_size:
            total += batch_size
            step += 1
            loss, correct = train_step(model, loss_fn, optimizer, inputs, target)
            train_losses.append(loss)
            train_accuracies.append(correct)

        if step == train_loader.steps_per_epoch:
            step = 0
            epoch += 1
            acc = sum(train_accuracies) / total
            total = 0
            print("Ep. {} - Loss: {:.3f}. Acc: {:.3f}".format(epoch, np.mean(train_losses), acc))
            train_losses=[]
            train_accuracies=[]

    test_losses=[]
    test_accuracies=[]

    for inputs, target in test_loader:
        pred=model(inputs, training=False)
        test_losses.append(loss_fn(target, pred))
        acc=(pred == target).numpy().astype(int)
        test_accuracies.append(acc)

    print("Done. Test loss: {:.3f}. Test acc: {:.3f}".format(
        np.mean(test_losses), np.mean(test_accuracies)))

    return train_losses, train_accuracies, test_losses, test_accuracies


In [470]:
#data = FakenNewsNet("./dataset/FakeNewsNet/politifact/")
data = PolitiFact()
#data = WICO()

In [382]:
y.shape

(1, 67, 67)